Nesta fase, irei iniciar a etapa de experimentação, onde irei testar uma série 
de modelos de classificação para encontrar o que melhor se adequa ao problema.
Vale ressaltar que apenas algumas colunas serão utilizadas para a criação dos
modelos, pois algumas só são obtidas ao fim do processo do SISU, como é o caso
da coluna "NOTA_CORTE" e "CLASSIFICACAO". Outras colunas como códigos das IES
e cursos também não serão utilizadas, pois não são relevantes para o problema.
Ao final, além do modelo, um conjunto de dados no formato .db será gerado para
ser consumido pelo app final. Segue abaixo as colunas que serão utilizadas:

- Modelo: IES, UF_CAMPUS, MUNICIPIO_CAMPUS, NOME_CURSO, GRAU, TURNO, 
TIPO_MOD_CONCORRENCIA, QT_VAGAS_CONCORRENCIA, PERCENTUAL_BONUS, PESO_L, PESO_CH,
PESO_CN, PESO_M, PESO_R, NOTA_MINIMA_L, NOTA_MINIMA_CH, NOTA_MINIMA_CN, 
NOTA_MINIMA_M, NOTA_MINIMA_R, MEDIA_MINIMA, OPCAO, NOTA_L, NOTA_CH, NOTA_CN, 
NOTA_M, NOTA_R, NOTA_L_COM_PESO, NOTA_CH_COM_PESO, NOTA_CN_COM_PESO, 
NOTA_M_COM_PESO, NOTA_R_COM_PESO, NOTA_CANDIDATO e APROVADO.

Vale ressaltar que parte das informações que serão utilizadas no Web App serão 
buscadas nos dados do SISU, como é o caso da QT_VAGAS_CONCORRENCIA, que é um 
valor que a universidade define para cada curso e não o usuário. Outras serão 
calculadas manualmente, como no caso das notas com peso.


In [1]:
import mlflow
import optuna
import pandas as pd
import category_encoders as ce

# Preprocessing & Models
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


# Métricas
from sklearn.metrics import log_loss

In [2]:
# Lendo os dados
dados_sisu = pd.read_parquet('../data/processed/dados_transformados.parquet/')

In [3]:
# Definindo as colunas que serão utilizadas para treinar o modelo
colunas_para_buscar = ['IES', 'UF_CAMPUS', 'MUNICIPIO_CAMPUS', 'NOME_CURSO', 
                       'GRAU', 'TURNO', 'TIPO_MOD_CONCORRENCIA', 
                       'QT_VAGAS_CONCORRENCIA', 'PERCENTUAL_BONUS', 'PESO_L', 
                       'PESO_CH', 'PESO_CN', 'PESO_M', 'PESO_R', 
                       'NOTA_MINIMA_L', 'NOTA_MINIMA_CH', 'NOTA_MINIMA_CN', 
                       'NOTA_MINIMA_M', 'NOTA_MINIMA_R', 'MEDIA_MINIMA', 
                       'OPCAO', 'NOTA_L', 'NOTA_CH', 'NOTA_CN', 'NOTA_M', 
                       'NOTA_R', 'NOTA_L_COM_PESO', 'NOTA_CH_COM_PESO', 
                       'NOTA_CN_COM_PESO', 'NOTA_M_COM_PESO', 'NOTA_R_COM_PESO',
                       'NOTA_CANDIDATO', 'APROVADO']

In [4]:
# Filtrando as colunas
dados_sisu = dados_sisu[colunas_para_buscar]

In [5]:
# Verificando o balanceamento da variável alvo
dados_sisu['APROVADO'].value_counts(normalize=True)

N    0.898245
S    0.101755
Name: APROVADO, dtype: float64

In [6]:
# Define o local para salvar os experimentos
mlflow.set_tracking_uri('../mlruns')

# Criando/acessando o experimento
mlflow.set_experiment('Comparando modelos')

<Experiment: artifact_location='/workspaces/sisu_analysis/notebooks/../mlruns/552820451131722872', creation_time=1701886234717, experiment_id='552820451131722872', last_update_time=1701886234717, lifecycle_stage='active', name='Comparando modelos', tags={}>

In [7]:
# Dividindo os dados em variaveis explicativas e variavel alvo
x = dados_sisu.drop(columns=['APROVADO'])
y = dados_sisu['APROVADO'].map({'S': 1, 'N': 0})

# Dividindo os dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.45, random_state=42, stratify=y)

# Dividindo os dados em teste e dev
x_teste, x_dev, y_teste, y_dev = train_test_split(x_teste, y_teste, test_size=0.5, random_state=42, stratify=y_teste)

# Dividindo os dados em dev e calibração
x_dev, x_calib, y_dev, y_calib = train_test_split(x_dev, y_dev, test_size=0.5, random_state=42, stratify=y_dev)


In [8]:
# Criando um scaler padrão
scale = y_treino.value_counts()[0] / y_treino.value_counts()[1] 

# Criando dicionário com os modelos
dict_models_scale_sensitive_cw = {"LR": LogisticRegression(random_state=200, 
                                                           class_weight='balanced')}

dict_models_scale_sensitive_no_cw = {"LR": LogisticRegression(random_state=200)}

dict_models_tree_based_cw = {"LGBM": LGBMClassifier(is_unbalance=True,
                                                 random_state=200),
                          "XGB": XGBClassifier(scale_pos_weight=scale,
                                               random_state=200),
                          "CTBC": CatBoostClassifier(auto_class_weights='Balanced',
                                                     random_state=200)}

dict_models_tree_based_no_cw = {"LGBM": LGBMClassifier(random_state=200),
                          "XGB": XGBClassifier(random_state=200),
                          "CTBC": CatBoostClassifier(random_state=200)}

# Criando dicionário com os encoders
dict_encoders = {"OHE": OneHotEncoder(drop='first'),
                 "TE": ce.TargetEncoder(),
                 "BE": ce.BinaryEncoder(),
                 "ME": ce.MEstimateEncoder(),
                 "WOE": ce.WOEEncoder(),
                 "CE": ce.CatBoostEncoder(),
                 "GE":ce.GrayEncoder()}

dict_scalers = {"SS": StandardScaler()}

# Criando dicionário com os transformers
dict_transformers = {"PT": PowerTransformer()}

In [9]:
# Definindo as folds
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=200)

# Retornando os nomes das colunas com mais de 25 valores únicos
cat_cols = x_treino.select_dtypes(include='object').columns
high_dim_cols = cat_cols[x_treino[cat_cols].nunique() > 25]

# Retornando os nomes das colunas com menos de 25 valores únicos
cat_cols = [col for col in cat_cols if col not in high_dim_cols]

# Buscando as colunas numéricas
num_cols = x_treino.select_dtypes(include=['int', 'float']).columns

In [10]:
## Iniciando os experimentos sem transformers e com class_weight
#for tag, model in dict_models_scale_sensitive_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#        for tag_scaler, scaler in dict_scalers.items():
#            
#            # Gerando a tag de identificação do modelo
#            nome_modelo = f'{tag}_CW_{tag_encoder}_{tag_scaler}'
#            
#            with mlflow.start_run(run_name=nome_modelo):
#                 
#                 # Criando os pipeline com os transformers
#                 pipe_cat = Pipeline([('encoder', encoder)])
#                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                 pipe_num = Pipeline([('scaler', scaler)])
#                 
#                 # Criando o transformador
#                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                 ('num', pipe_num, num_cols),
#                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
#                 
#                 # Criando o pipeline final
#                 pipe = Pipeline([('transformer', transformer),
#                                 ('model', model)])
#                 
#                 # Executando o cross validation
#                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#                 
#                 # Calculando a média das métricas
#                 mean_score = cross_val_scores.mean()           
#                 
#                 # Salvando a métrica da folder 1
#                 mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#                 
#                 # Salvando a métrica da folder 2
#                 mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#                
#                 # Salvando a métrica da folder 3
#                 mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#                
#                 # Salvando a métrica da folder 4
#                 mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#                
#                 # Salvando a métrica da folder 5
#                 mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#                 
#                 # Salvando as métricas
#                 mlflow.log_metric('log_loss_mean', mean_score)

In [11]:
## Iniciando os experimentos sem transformers e sem class_weight
#for tag, model in dict_models_scale_sensitive_no_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#        for tag_scaler, scaler in dict_scalers.items():
#            
#            # Gerando a tag de identificação do modelo
#            nome_modelo = f'{tag}_NO_CW_{tag_encoder}_{tag_scaler}'
#            
#            with mlflow.start_run(run_name=nome_modelo):
#                 
#                 # Criando os pipeline com os transformers
#                 pipe_cat = Pipeline([('encoder', encoder)])
#                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                 pipe_num = Pipeline([('scaler', scaler)])
#                 
#                 # Criando o transformador
#                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                 ('num', pipe_num, num_cols),
#                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
#                 
#                 # Criando o pipeline final
#                 pipe = Pipeline([('transformer', transformer),
#                                 ('model', model)])
#                 
#                 # Executando o cross validation
#                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#                 
#                 # Calculando a média das métricas
#                 mean_score = cross_val_scores.mean()           
#                 
#                 # Salvando a métrica da folder 1
#                 mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#                 
#                 # Salvando a métrica da folder 2
#                 mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#                
#                 # Salvando a métrica da folder 3
#                 mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#                
#                 # Salvando a métrica da folder 4
#                 mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#                
#                 # Salvando a métrica da folder 5
#                 mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#                 
#                 # Salvando as métricas
#                 mlflow.log_metric('log_loss_mean', mean_score)

In [12]:
## Iniciando os experimentos com transformers e sem class_weight
#for tag, model in dict_models_scale_sensitive_no_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#        for tag_scaler, scaler in dict_scalers.items():
#            for tag_transformer, transformer in dict_transformers.items():
#            
#                # Gerando a tag de identificação do modelo
#                nome_modelo = f'{tag}_NO_CW_{tag_encoder}_{tag_scaler}_{tag_transformer}'
#
#                with mlflow.start_run(run_name=nome_modelo):
#
#                     # Criando os pipeline com os transformers
#                     pipe_cat = Pipeline([('encoder', encoder)])
#                     pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                     pipe_num = Pipeline([('scaler', scaler),
#                                          ('transformer', transformer)])
#
#                     # Criando o transformador
#                     transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                     ('num', pipe_num, num_cols),
#                                                     ('high_dim', pipe_high_dim, high_dim_cols)])
#
#                     # Criando o pipeline final
#                     pipe = Pipeline([('transformer', transformer),
#                                     ('model', model)])
#
#                     # Executando o cross validation
#                     cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#
#                     # Calculando a média das métricas
#                     mean_score = cross_val_scores.mean()         
#
#                     # Salvando a métrica da folder 1
#                     mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#
#                     # Salvando a métrica da folder 2
#                     mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#
#                     # Salvando a métrica da folder 3
#                     mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#
#                     # Salvando a métrica da folder 4
#                     mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#
#                     # Salvando a métrica da folder 5
#                     mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#
#                     # Salvando as métricas
#                     mlflow.log_metric('log_loss_mean', mean_score)

In [13]:
## Iniciando os experimentos com transformers e com class_weight
#for tag, model in dict_models_scale_sensitive_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#        for tag_scaler, scaler in dict_scalers.items():
#            for tag_transformer, transformer in dict_transformers.items():
#            
#                # Gerando a tag de identificação do modelo
#                nome_modelo = f'{tag}_CW_{tag_encoder}_{tag_scaler}_{tag_transformer}'
#
#                with mlflow.start_run(run_name=nome_modelo):
#
#                     # Criando os pipeline com os transformers
#                     pipe_cat = Pipeline([('encoder', encoder)])
#                     pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                     pipe_num = Pipeline([('scaler', scaler),
#                                          ('transformer', transformer)])
#
#                     # Criando o transformador
#                     transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                     ('num', pipe_num, num_cols),
#                                                     ('high_dim', pipe_high_dim, high_dim_cols)])
#
#                     # Criando o pipeline final
#                     pipe = Pipeline([('transformer', transformer),
#                                     ('model', model)])
#
#                     # Executando o cross validation
#                     cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#
#                     # Calculando a média das métricas
#                     mean_score = cross_val_scores.mean()         
#
#                     # Salvando a métrica da folder 1
#                     mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#
#                     # Salvando a métrica da folder 2
#                     mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#
#                     # Salvando a métrica da folder 3
#                     mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#
#                     # Salvando a métrica da folder 4
#                     mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#
#                     # Salvando a métrica da folder 5
#                     mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#
#                     # Salvando as métricas
#                     mlflow.log_metric('log_loss_mean', mean_score)

In [14]:
## Iniciando os experimentos sem transformers e com class_weight
#for tag, model in dict_models_tree_based_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#            
#            # Gerando a tag de identificação do modelo
#            nome_modelo = f'{tag}_CW_{tag_encoder}'
#            
#            with mlflow.start_run(run_name=nome_modelo):
#                 
#                 # Criando os pipeline com os transformers
#                 pipe_cat = Pipeline([('encoder', encoder)])
#                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                 
#                 # Criando o transformador
#                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
#                 
#                 # Criando o pipeline final
#                 pipe = Pipeline([('transformer', transformer),
#                                 ('model', model)])
#                 
#                 # Executando o cross validation
#                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#                 
#                 # Calculando a média das métricas
#                 mean_score = cross_val_scores.mean()         
#                 
#                 # Salvando a métrica da folder 1
#                 mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#                 
#                 # Salvando a métrica da folder 2
#                 mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#                
#                 # Salvando a métrica da folder 3
#                 mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#                
#                 # Salvando a métrica da folder 4
#                 mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#                
#                 # Salvando a métrica da folder 5
#                 mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#                 
#                 # Salvando as métricas
#                 mlflow.log_metric('log_loss_mean', mean_score)
#

In [15]:
## Iniciando os experimentos sem transformers e sem class_weight
#for tag, model in dict_models_tree_based_no_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#            
#            # Gerando a tag de identificação do modelo
#            nome_modelo = f'{tag}_NO_CW_{tag_encoder}'
#            
#            with mlflow.start_run(run_name=nome_modelo):
#                 
#                 # Criando os pipeline com os transformers
#                 pipe_cat = Pipeline([('encoder', encoder)])
#                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                 
#                 # Criando o transformador
#                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
#                 
#                 # Criando o pipeline final
#                 pipe = Pipeline([('transformer', transformer),
#                                 ('model', model)])
#                 
#                 # Executando o cross validation
#                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#                 
#                 # Calculando a média das métricas
#                 mean_score = cross_val_scores.mean()         
#                 
#                 # Salvando a métrica da folder 1
#                 mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#                 
#                 # Salvando a métrica da folder 2
#                 mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#                
#                 # Salvando a métrica da folder 3
#                 mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#                
#                 # Salvando a métrica da folder 4
#                 mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#                
#                 # Salvando a métrica da folder 5
#                 mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#                 
#                 # Salvando as métricas
#                 mlflow.log_metric('log_loss_mean', mean_score)
#

In [16]:
# Especificando as colunas para retornar
colunas = ['tags.mlflow.runName', 'metrics.log_loss_mean']

# Buscando os melhores modelos
mlflow.search_runs(order_by=['metrics.log_loss_mean DESC'], max_results=15)[colunas]

,tags.mlflow.runName,metrics.log_loss_mean
0,LR_NO_CW_BE_SS,-0.263045
1,LR_NO_CW_BE_SS_PT,-0.263931
2,LR_NO_CW_GE_SS,-0.265065
3,LR_NO_CW_GE_SS_PT,-0.265762
4,LR_NO_CW_OHE_SS_PT,-0.265789
5,LR_NO_CW_OHE_SS,-0.266268
6,LR_NO_CW_WOE_SS,-0.270550
7,LR_NO_CW_TE_SS,-0.270770
8,LR_NO_CW_ME_SS,-0.270770
9,LR_NO_CW_CE_SS,-0.270770


Os modelos sem Class_Weight foram os melhores classificadores. Dentre os 
melhores, é perceptível a baixa diferença entre os modelos. Dado a isso, iremos
selecionar os dois melhores modelos para a tunagem de hiperparâmetros, que são 
a Regressão Logística e o CatBoost. Os dois serão testados em um ensemble.

Como a natureza da solução exige respostas rápidas, a Regressão Logística terá 
preferência sobre o CatBoost e o Ensemble de ambos.

### Regressão Logística

In [17]:
## Criando função para tunar o modelo
#def objective(trial):
#
#    params = {
#        'C': trial.suggest_float('C', 1e-4, 1e+4, log=True),
#        'penalty': trial.suggest_categorical('penalty', [None, 'l2']),
#        'solver': trial.suggest_categorical('solver', ['lbfgs', 'saga', 'newton-cholesky']),
#        'max_iter': trial.suggest_int('max_iter', 50, 1000),
#        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
#        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
#        'random_state': 200
#    }
#    
#    # Criando os pipeline com os transformers
#    pipe_cat = Pipeline([('encoder', ce.BinaryEncoder())])
#    pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#    pipe_num = Pipeline([('scaler', StandardScaler())])
#    
#    # Criando o transformador
#    transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                    ('num', pipe_num, num_cols),
#                                    ('high_dim', pipe_high_dim, high_dim_cols)])
#    
#    # Criando o pipeline final
#    pipe = Pipeline([('transformer', transformer),
#                    ('model', LogisticRegression(**params))])
#
#    # Treinando o modelo com os dados de treino
#    pipe.fit(x_treino, y_treino)
#   
#    logloss = log_loss(y_dev, pipe.predict_proba(x_dev))
#    
#    return logloss
#
## Criando o estudo de otimização
#study = optuna.create_study(direction = 'minimize')
#study.optimize(objective, n_trials = 15)

[I 2023-12-07 18:52:18,642] A new study created in memory with name: no-name-efe560b1-4cc6-4caa-a2e2-b359ce6e04fc
/workspaces/sisu_analysis/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2023-12-07 18:56:07,592] Trial 0 finished with value: 0.6076452777922804 and parameters: {'C': 0.3491528335036483, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 232, 'fit_intercept': False, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.6076452777922804.
/workspaces/sisu_analysis/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2023-12-07 18:56:20,501] Trial 1 finished with value: 0.267158642020306 and parameters: {'C': 270.48268823332904, 'penalty': None, 'solver': 'lbfgs', 'max_iter': 145, 'fit_intercept': False, 'class_weight': None}. Best is tr

In [31]:
# O resultado do hiperparâmetro otimizado pode ser diferente a cada execução.
# Resultado atingido: 0.23

# Checando os melhores parâmetros
#study.best_params

In [28]:
# Criando os pipeline com os transformers
pipe_cat = Pipeline([('encoder', ce.BinaryEncoder())])
pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
pipe_num = Pipeline([('scaler', StandardScaler())])

# Criando o transformador
transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
                                ('num', pipe_num, num_cols),
                                ('high_dim', pipe_high_dim, high_dim_cols)])

# Criando o pipeline final
best_lr = Pipeline([('transformer', transformer),
                ('model', LogisticRegression(C=0.08250109742237544,
                                             penalty='l2',
                                             solver='newton-cholesky',
                                             max_iter=244,
                                             fit_intercept=False,
                                             class_weight=None,
                                             random_state=200))])

best_lr.fit(x_treino, y_treino)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   BinaryEncoder())]),
                                                  ['GRAU', 'TURNO',
                                                   'TIPO_MOD_CONCORRENCIA',
                                                   'PESO_L', 'PESO_CH',
                                                   'PESO_CN', 'PESO_M',
                                                   'PESO_R', 'OPCAO']),
                                                 ('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['QT_VAGAS_CONCORRENCIA', 'PERCENTUAL_BONUS', 'NOTA_MINIMA_L',
       'NOTA_MINIMA_CH', '...
       'NOTA_M_COM_PESO', 'NOTA_R_COM_PESO', 'NOTA_CANDIDATO'],
      dtype='object')),
                                                 ('high_dim',
                                                  Pipeline(steps=[('encoder',
                                                                   CountEncoder(combine_min_nan_groups=True))]),
                                                  Index(['IES', 'UF_CAMPUS', 'MUNICIPIO_CAMPUS', 'NOME_CURSO'], dtype='object'))])),
                ('model',
                 LogisticRegression(C=0.08250109742237544, fit_intercept=False,
                                    max_iter=244, random_state=200,
                                    solver='newton-cholesky'))])

### CatBoost

In [30]:
## Criando função para tunar o modelo
#def objective(trial):
#
#    params = {
#        'objective': 'Logloss',
#        'eval_metric': 'Logloss',
#        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
#        'depth': trial.suggest_int('depth', 3, 10),
#        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
#        'subsample': trial.suggest_float('subsample', 0.5, 1),
#        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1),
#        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10),
#        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
#        'random_state': 200
#    }
#    
#    pipe_cat = Pipeline([('encoder', ce.GrayEncoder())])
#    pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#
#    # Criando o transformador
#    transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                    ('high_dim', pipe_high_dim, high_dim_cols)])
#
#    # Criando o pipeline final
#    pipe = Pipeline([('transformer', transformer),
#                    ('rf', CatBoostClassifier(**params))])
#
#    # Treinando o modelo com os dados de treino
#    pipe.fit(x_treino, y_treino)
#   
#    logloss = log_loss(y_dev, pipe.predict_proba(x_dev))
#    
#    return logloss
#
## Criando o estudo de otimização
#study = optuna.create_study(direction = 'minimize')
#study.optimize(objective, n_trials = 15)

[I 2023-12-07 19:16:40,494] A new study created in memory with name: no-name-e83bb914-65d4-4d46-a894-3d522794d4a1


0:	learn: 0.6476222	total: 174ms	remaining: 2m 53s
1:	learn: 0.6098738	total: 259ms	remaining: 2m 9s
2:	learn: 0.5782618	total: 347ms	remaining: 1m 55s
3:	learn: 0.5517062	total: 438ms	remaining: 1m 49s
4:	learn: 0.5312711	total: 531ms	remaining: 1m 45s
5:	learn: 0.5135467	total: 621ms	remaining: 1m 42s
6:	learn: 0.4996119	total: 718ms	remaining: 1m 41s
7:	learn: 0.4881099	total: 810ms	remaining: 1m 40s
8:	learn: 0.4780260	total: 902ms	remaining: 1m 39s
9:	learn: 0.4699675	total: 991ms	remaining: 1m 38s
10:	learn: 0.4632087	total: 1.08s	remaining: 1m 37s
11:	learn: 0.4574896	total: 1.17s	remaining: 1m 36s
12:	learn: 0.4524741	total: 1.27s	remaining: 1m 36s
13:	learn: 0.4482609	total: 1.36s	remaining: 1m 35s
14:	learn: 0.4450364	total: 1.45s	remaining: 1m 35s
15:	learn: 0.4417377	total: 1.54s	remaining: 1m 35s
16:	learn: 0.4391160	total: 1.63s	remaining: 1m 34s
17:	learn: 0.4365027	total: 1.72s	remaining: 1m 34s
18:	learn: 0.4341332	total: 1.82s	remaining: 1m 34s
19:	learn: 0.4324779	to

[I 2023-12-07 19:18:21,032] Trial 0 finished with value: 0.2994706230366258 and parameters: {'scale_pos_weight': 1.946917166950045, 'depth': 7, 'min_child_samples': 3, 'subsample': 0.963250533239097, 'colsample_bylevel': 0.5575870684991423, 'reg_lambda': 8.693374908645778, 'learning_rate': 0.07820772969101444}. Best is trial 0 with value: 0.2994706230366258.


0:	learn: 0.6899778	total: 54.3ms	remaining: 54.3s
1:	learn: 0.6865409	total: 110ms	remaining: 54.9s
2:	learn: 0.6835903	total: 167ms	remaining: 55.4s
3:	learn: 0.6805316	total: 229ms	remaining: 57s
4:	learn: 0.6779316	total: 303ms	remaining: 1m
5:	learn: 0.6753097	total: 411ms	remaining: 1m 8s
6:	learn: 0.6726434	total: 470ms	remaining: 1m 6s
7:	learn: 0.6700961	total: 529ms	remaining: 1m 5s
8:	learn: 0.6676040	total: 585ms	remaining: 1m 4s
9:	learn: 0.6655080	total: 639ms	remaining: 1m 3s
10:	learn: 0.6630790	total: 698ms	remaining: 1m 2s
11:	learn: 0.6610455	total: 760ms	remaining: 1m 2s
12:	learn: 0.6588544	total: 826ms	remaining: 1m 2s
13:	learn: 0.6569086	total: 884ms	remaining: 1m 2s
14:	learn: 0.6551649	total: 946ms	remaining: 1m 2s
15:	learn: 0.6534985	total: 1.01s	remaining: 1m 2s
16:	learn: 0.6520063	total: 1.07s	remaining: 1m 1s
17:	learn: 0.6504890	total: 1.13s	remaining: 1m 1s
18:	learn: 0.6487740	total: 1.19s	remaining: 1m 1s
19:	learn: 0.6473745	total: 1.25s	remaining: 

[I 2023-12-07 19:19:29,260] Trial 1 finished with value: 0.5249203389222242 and parameters: {'scale_pos_weight': 7.144454796693722, 'depth': 4, 'min_child_samples': 7, 'subsample': 0.5387070162286863, 'colsample_bylevel': 0.7172778419785718, 'reg_lambda': 8.903789627731983, 'learning_rate': 0.02480673985781702}. Best is trial 0 with value: 0.2994706230366258.


...

[I 2023-12-07 19:36:28,938] Trial 12 finished with value: 0.2929045759095017 and parameters: {'scale_pos_weight': 1.576944884480049, 'depth': 5, 'min_child_samples': 13, 'subsample': 0.9820969726200324, 'colsample_bylevel': 0.9846208220325542, 'reg_lambda': 6.0614631042340905, 'learning_rate': 0.0390497672673001}. Best is trial 2 with value: 0.2830682937035806.


0:	learn: 0.6565755	total: 67.2ms	remaining: 1m 7s
1:	learn: 0.6240340	total: 136ms	remaining: 1m 8s
2:	learn: 0.5946047	total: 207ms	remaining: 1m 8s
3:	learn: 0.5683271	total: 304ms	remaining: 1m 15s
4:	learn: 0.5448624	total: 418ms	remaining: 1m 23s
5:	learn: 0.5241701	total: 518ms	remaining: 1m 25s
6:	learn: 0.5054651	total: 605ms	remaining: 1m 25s
7:	learn: 0.4896747	total: 682ms	remaining: 1m 24s
8:	learn: 0.4738612	total: 759ms	remaining: 1m 23s
9:	learn: 0.4605696	total: 841ms	remaining: 1m 23s
10:	learn: 0.4481678	total: 912ms	remaining: 1m 22s
11:	learn: 0.4365240	total: 988ms	remaining: 1m 21s
12:	learn: 0.4257426	total: 1.06s	remaining: 1m 20s
13:	learn: 0.4161763	total: 1.13s	remaining: 1m 19s
14:	learn: 0.4074654	total: 1.2s	remaining: 1m 18s
15:	learn: 0.3995004	total: 1.27s	remaining: 1m 18s
16:	learn: 0.3923802	total: 1.34s	remaining: 1m 17s
17:	learn: 0.3863855	total: 1.42s	remaining: 1m 17s
18:	learn: 0.3807993	total: 1.5s	remaining: 1m 17s
19:	learn: 0.3754478	total

[I 2023-12-07 19:37:52,558] Trial 13 finished with value: 0.28412938779799246 and parameters: {'scale_pos_weight': 1.1053092681742818, 'depth': 5, 'min_child_samples': 15, 'subsample': 0.8655341168057921, 'colsample_bylevel': 0.6337703602552688, 'reg_lambda': 9.599245044680334, 'learning_rate': 0.04015203111073107}. Best is trial 2 with value: 0.2830682937035806.


0:	learn: 0.6834177	total: 69.7ms	remaining: 1m 9s
1:	learn: 0.6746220	total: 140ms	remaining: 1m 10s
2:	learn: 0.6656862	total: 258ms	remaining: 1m 25s
3:	learn: 0.6567852	total: 332ms	remaining: 1m 22s
4:	learn: 0.6492813	total: 400ms	remaining: 1m 19s
5:	learn: 0.6417207	total: 479ms	remaining: 1m 19s
6:	learn: 0.6345323	total: 549ms	remaining: 1m 17s
7:	learn: 0.6279609	total: 620ms	remaining: 1m 16s
8:	learn: 0.6215722	total: 693ms	remaining: 1m 16s
9:	learn: 0.6156941	total: 766ms	remaining: 1m 15s
10:	learn: 0.6101135	total: 835ms	remaining: 1m 15s
11:	learn: 0.6049031	total: 905ms	remaining: 1m 14s
12:	learn: 0.5998851	total: 973ms	remaining: 1m 13s
13:	learn: 0.5951622	total: 1.04s	remaining: 1m 13s
14:	learn: 0.5904824	total: 1.12s	remaining: 1m 13s
15:	learn: 0.5860408	total: 1.19s	remaining: 1m 13s
16:	learn: 0.5819130	total: 1.26s	remaining: 1m 12s
17:	learn: 0.5780564	total: 1.33s	remaining: 1m 12s
18:	learn: 0.5741763	total: 1.41s	remaining: 1m 12s
19:	learn: 0.5707590	t

[I 2023-12-07 19:39:14,670] Trial 14 finished with value: 0.34483423450560774 and parameters: {'scale_pos_weight': 2.9037919555069944, 'depth': 5, 'min_child_samples': 10, 'subsample': 0.8481145066472896, 'colsample_bylevel': 0.6306372349949931, 'reg_lambda': 9.969297091677698, 'learning_rate': 0.023893554449801166}. Best is trial 2 with value: 0.2830682937035806.


Não foi registrado um aumento significativa com a tunagem, por isso, o modelo
será o padrão.